In [1]:
import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import CountVectorizer # Bag of Words
from sklearn.svm import SVC 
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

import warnings
warnings.filterwarnings("ignore")

#### Importando os dados

In [2]:
data = pd.read_table("data/data_estag_ds.tsv")

In [3]:
data.head()

,ID,TITLE
0,1041354,Acessório T - Jean Bag For Girls para DS Lite
1,1041782,Carrinho de Bebê Berço-Passeio - Pegasus Pink ...
2,1041834,Carrinho de Bebê para Gêmeos Berço-Passeio - T...
3,1042568,Car Center - Calesita
4,1042584,Donka Trem com Som - Calesita


#### Após uma breve análise do arquivo de dados, é possível perceber a presença de smartphones logo nos primeiros exemplos, porém, eles só aparecem novamente no final do arquivo. Graças a essa separação clara, é fácil classificar manualmente os exemplos para usar aprendizado supervisionado

In [4]:
data["CLASS"] = np.zeros(len(data)) # Adiciona uma coluna "CLASS" somente com valores 0

# Adiciona 1 a classe dos exemplos que sao smartphones
data.loc[6:39, "CLASS"] = 1
data.loc[2035:, "CLASS"] = 1

total_classes = data["CLASS"].value_counts()
print(total_classes)
print("\nPorcentagem de Smartphones", total_classes[1]/(total_classes[0]+total_classes[1]))

0.0    2001
1.0     605
Name: CLASS, dtype: int64

Porcentagem de Smartphones 0.23215656178050653


In [5]:
data.isnull().sum() # Nao tem dados faltando

ID       0
TITLE    0
CLASS    0
dtype: int64

In [6]:
data[data['CLASS'] == 1].head() # Dataset atualizado

,ID,TITLE,CLASS
6,1940486,Smartphone LG Optimus L7 II Dual P716 Preto co...,1.0
7,2277815,Smartphone Samsung Galaxy S4 Mini Duos Preto c...,1.0
8,2961097,Smartphone Samsung Galaxy S5 SM-G900M Branco ...,1.0
9,2961099,Smartphone Samsung Galaxy S5 SM-G900M Dourado...,1.0
10,3031744,Smartphone LG L90 Dual D410 Preto com Tela de ...,1.0


In [7]:
# Faz uma bag of words com matriz esparsa
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(data.loc[:,"TITLE"])
y = data.loc[:,"CLASS"]

print("X:", X.toarray().shape)
print("y:",y.shape)
print(type(X))

X: (2606, 4165)
y: (2606,)
<class 'scipy.sparse.csr.csr_matrix'>


In [8]:
# Separando os dados de teste e de treino
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)

# Garantindo que haja uma quantidade razoável de exemplos positivos em ambos os conjuntos
n_y_train = len(y_train[y_train == 1])
n_y_test = len(y_test[y_test == 1])

print("Porcentagem de Smartphones no conjunto de treino:", n_y_train/len(y_train))
print("Porcentagem de Smartphones no conjunto de teste:", n_y_test/len(y_test))

Porcentagem de Smartphones no conjunto de treino: 0.23739035087719298
Porcentagem de Smartphones no conjunto de teste: 0.21994884910485935


#### Aplicando SVM com Kernel Linear

In [9]:
clf = SVC(kernel='linear', random_state=1)
clf.fit(X_train, y_train)
print("Accuracy:\n", clf.score(X_test,y_test))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, clf.predict(X_test)))

Accuracy:
 0.9936061381074168

Confusion Matrix:
 [[607   3]
 [  2 170]]


In [10]:
# Adiciona à coluna CLASS os valores preditos pelo classificador
data.loc[:,"CLASS"] = clf.predict(X)

In [11]:
# 20 exemplos de smartphone
data[data["CLASS"] == 1].tail(20)

,ID,TITLE,CLASS
2586,1106549255,Celular Samsung J400m Galaxy J4 Violeta 32 Gb,1.0
2587,1106609097,Celular Samsung J400m Galaxy J4 Prata 32 Gb,1.0
2588,1107590289,Celular Xiaomin Redmi Note 5 64gb 4g Ram Pront...,1.0
2589,1109394935,Iphone 8 64 Gb Apple Desbloqueado Anatel Lacra...,1.0
2590,1109397175,Iphone 8 64 Gb Apple Desbloqueado Anatel Lacra...,1.0
2591,1110226459,Apple Iphone 8 Plus 64gb Gold Dourado - Novo L...,1.0
2592,1111104349,Smartphone Xiaomi Redmi Note 4x 64gb 4gb Rom G...,1.0
2593,1112728925,Celular Xiaomi Redmi S2 64gb 4gb De Ram 4g Lac...,1.0
2594,1112883328,Samsung Galaxy J7 Prime 2 Tv,1.0
2595,1113203077,Smartphone Samsung Galaxy J6 Sm-j600g/ds Dual ...,1.0


In [12]:
# Salva os resultados em formato xlsx (excel) e csv

data[["TITLE","CLASS"]].to_excel("data/resultado.xlsx")
data[["TITLE","CLASS"]].to_csv("data/resultado.csv")

# Análises Finais:
- Em minha primeira tentativa, usei a abordagem não supervisionada (K-Means), pois achei que não seria permitido classificar manualmente os dados, já que a feature target não foi fornecida. Isso causou vários problemas como:
<br>
-- Impossibilidade de checar automaticamente a performance do classificador;
<br>
-- Necessidade de checagem manual para cada um dos clusteres;
<br>
-- Performance notávelmente abalada;
<br>
-- etc.
<br>
<br>
- Nesta nova tentativa, usei a dica que recebi na entrevista e usei a matriz esparsa para armazenar a bag of words. Além disso, abri mão de algumas partes do código que havia implementado sozinho, utilizando então bibliotecas mais otimizadas. Isso aumentou muito o desempenho do código. 
<br>
<br>
- Sobre o classificador, resolvi usar o SVM com kernel linear. Tentei usar o 'default' com kernel RBF, mas o desempenho não foi bom. Talvez o kernel rbf complique muito o modelo, causando overfit. Já com o SVM Linear, o desempenho foi muito bom, acertando praticamente todos os casos.